In [5]:
# !pip install --upgrade pip

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import tensorflow as tf
import json

# from scipy.special import entr
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer

In [6]:
CORES = -1
SEED = 567
FINAL_EXP_RESULTS_PATH="final_experiments_results"
print(f"\x1b[32mSEED: {SEED}\x1b[0m")

# set up macros
EXP_NUM_0=0
EXP_NUM_1_1=1.1
EXP_NUM_1_2=1.2
EXP_NUM_2_1=2.1
EXP_NUM_2_2=2.2
EXP_NUM_3_1=3.1
EXP_NUM_3_2=3.2
EXP_NUM_4_1=4.1
EXP_NUM_4_2=4.2
EXP_NUM_5_1=5.1
EXP_NUM_5_2=5.2
EXP_NUM_6=6

# to be added to the seed when reseting the np and tf
EXP_NUM_2=2
EXP_NUM_5=5

CUT_OFF_FREQ_RANGE=list(range(1, 50, 4))
EMA_SPAN_RANGE=list(range(1, 50, 4))
SMA_WINSIZE_RANGE=list(range(1, 50, 4))
OVERLAP_EXP_RANGE=list(range(1, 100, 4))


# FOR EXP 2 AND 5 ACROSS NOTEBOOKS
CHOICE_NUM_PAIRS=len(list(range(1, 50, 4)))

WINDOW_SIZE_LST = [125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
OVERLAP=0.5

FILTER_ORDER=10
SAMPLING_FREQ=100
FILTFILT=1

DASH_MACRO_NUM=-99999

SAVE_PATH_NAME="SCNN_trained_models"
CLASSIFIER_TYPE_LST=["OCSVM", "kNN", "IF", "LOF"]

TRAINING_CONFIG_DICT_FILE_NAME="model_archi_performance_lr_dict.json"
TRAINING_CONFIG_DICT_FOLDER_PATH="siamese_cnn_results_final"
with open(f"{TRAINING_CONFIG_DICT_FOLDER_PATH}/{TRAINING_CONFIG_DICT_FILE_NAME}", 'r') as file:
    TRAINING_CONFIG_DICT=json.load(file)
    


def init_experiment_params(exp_config, exp_num=0):
    
    np.random.seed(SEED+exp_num)
    tf.random.set_seed(SEED+exp_num)

    exp_config.SMA_per_win_winsize=None
    exp_config.EMA_per_win_span=None
    exp_config.Butter_per_win_argdict=None
    exp_config.cut_off_freq=None
    exp_config.winsize=None
    exp_config.span=None

    exp_config.filter_order = FILTER_ORDER
    exp_config.sampling_freq = SAMPLING_FREQ
    exp_config.filtfilt = FILTFILT

    print("reseting experiment params successful!")

SEED: 567


In [1]:
list(range(1, 50, 4))

[1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49]

['SCNN_1_3_conv_1_dense_arg_dict_default',
 'SCNN_3_123_conv_1_dense_arg_dict_default',
 'SCNN_3_1_conv_1_dense_arg_dict_default',
 'SCNN_1_2_conv_1_dense_arg_dict_default',
 'SCNN_1_1_conv_1_dense_arg_dict_default',
 'SCNN_1_2_conv_2_dense_arg_dict_default',
 'SCNN_1_5_conv_1_dense_arg_dict_default',
 'SCNN_4_1234_conv_1_dense_arg_dict_default']

In [16]:
MAGENTA = (202/255, 18/255, 125/255)

In [ ]:
METRIC_WINSIZE_TABLE_COLS = [("Window Size", 125), ("Window Size", 250), ("Window Size", 500), ("Window Size", 750), ("Window Size", 1000), 
                 ("Window Size", 1250), ("Window Size", 1500), ("Window Size", 1750), ("Window Size", 2000), ("", 'mean'), ("", 'variance')]
STYLER_ERR_FORMAT_DICT={key: "{:.4f}" for key in METRIC_WINSIZE_TABLE_COLS}
STYLER_IMPROVEMENT_FORMAT_DICT={key: "{:.2f}" for key in METRIC_WINSIZE_TABLE_COLS}

In [17]:
get_new_scaler_dict = {"StandardScaler": StandardScaler, "MinMaxScaler": MinMaxScaler, "Normalizer": Normalizer, 
                       "MaxAbsScaler": MaxAbsScaler, "RobustScaler": RobustScaler, "PowerTransformer": PowerTransformer}

sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 26, "axes.labelsize": 26, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 20, "ytick.labelsize": 20})

err_distro_rc={'figure.figsize':(20,10), 'figure.dpi': 180, "legend.fontsize": 30, "axes.labelsize": 30, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 25, "ytick.labelsize": 25}

In [18]:
sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize": 9, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
            "xtick.labelsize": 11, "ytick.labelsize": 11})
# sns.set(rc={'figure.figsize':(12,8), 'figure.dpi': 180, "legend.fontsize":11, "axes.labelsize": 11, #"xtick.fontsize": 26, "ytick.fontsize": 20, 
#             "xtick.labelsize": 11, "ytick.labelsize": 11})

sns.set_style("darkgrid")
sns.set_context("paper")
sns.set(font="sans")
sns.set_palette("tab10")

# for plain matplotlib:
plt.style.use(["seaborn-darkgrid", "seaborn-paper"])
plt.rc("font", family="sans", size=8)
plt.rc("axes", titlesize=6)
plt.rc("axes", labelsize=6)
plt.rc("xtick", labelsize=6)
plt.rc("ytick", labelsize=6)
plt.rc("xtick.major", pad=1)
plt.rc("ytick.major", pad=3)
plt.rc("legend", fontsize=6)
plt.rc("figure", titlesize=6)

In [19]:
# Define a style I use a lot for boxplots:
utils_boxplot_style = dict(
    color="tab:blue",
    linewidth=0.5,
    saturation=1,
    width=0.7,
    flierprops=dict(
        marker="o", markersize=2, markerfacecolor="none", markeredgewidth=0.5
    ),
)

# Define a style I use a lot for lineplots:
utils_lineplot_style = dict(
    color="tab:blue", linewidth=0.5, marker="o", markersize=3, markeredgewidth=0.5
)

In [32]:
print(20*'-' + "\x1b[32mSEED and CONSTANTS imported\x1b[0m" + 20*'-')

--------------------SEED and CONSTANTS imported--------------------
